In [29]:
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
from typing import List
import numpy as np
import sasc.generate_helper
import sasc.viz
from pprint import pprint
import pandas as pd
import joblib
from collections import defaultdict
from sasc.config import RESULTS_DIR, REPO_DIR, CACHE_DIR
from typing import Tuple
import sys
from functools import partial
import json

sys.path.append(join(REPO_DIR, "notebooks_stories", "0_voxel_select"))
subject = 'UTS02'

In [30]:
r = pd.read_pickle("../../results/results_fmri_full_1500.pkl").sort_values(
    by=["top_score_synthetic"], ascending=False
)
r["id"] = (
    "('"
    + r["top_explanation_init_strs"].str.replace(" ", "_").str.slice(stop=20)
    + "', '"
    + r["subject"]
    + "', "
    + r["module_num"].astype(str)
    + "),"
)
r['id_tup'] = r.apply(lambda row: (row['top_explanation_init_strs'].replace(" ", "_")[:20], row['subject'], row['module_num']), axis=1)
r = r[r.subject == subject]
r["std_score_synthetic"] = r["score_synthetic"].apply(np.std)
r["mean_score_synthetic"] = r["score_synthetic"].apply(np.mean)
r["explanation_init_strs"] = r.apply(
    lambda row: '\n----\n'.join(np.array(row["explanation_init_strs"])[np.array(row["score_synthetic"]) > 0.04]),
    axis=1,
)
r["score_synthetic"] = r.apply(
    lambda row: np.array(row["score_synthetic"])[np.array(row["score_synthetic"]) > 0.04].round(decimals=3), axis=1
)
r = r[r["score_synthetic"].apply(len) > 1]
r = r.sort_values(by="mean_score_synthetic", ascending=False)



with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 400
):
    display(
        r[["id", "explanation_init_strs", "mean_score_synthetic", "fmri_test_corr", 'score_synthetic']]
        .sort_values(by="mean_score_synthetic", ascending=False)
        .transpose()
        .round(3)
        .style
        .format(precision=3)
    )

,925,694,1498,805,1358,1113,654,27,1328,286,632,696,250,1146,911,739,934,1255,80,527,926,641,1273,472,750,1298,923,1301,1490,1011,1312,1088,748,798,711,1143,872,207,897,101,1230,1031,217,1373,1027,636,130,1384,746,1363,1058,883,1190,615,20,998,816,1128,1409,1221,819,451,596,842,375,1303,1116,34,1341,72,318,515,1446,786,1159,804,1187,147,744,781,1174,962,731,860,69,444,1458,1378,568,1454,257,213,662,30,680,909,771,247,391,253,64,440,824,1355,291,198,1215,9,56,1199,935,75,1023
id,"('a_specific_decade_in', 'UTS02', 271),","('death_and_saying_goo', 'UTS02', 155),","('language,_either_in_', 'UTS02', 449),","('physical_features,_s', 'UTS02', 217),","('body_language_and_fa', 'UTS02', 459),","('physical_actions,_su', 'UTS02', 220),","('describing_physical_', 'UTS02', 293),","('a_specific_year_in_t', 'UTS02', 412),","('a_specific_decade_in', 'UTS02', 274),","('counting_or_measurin', 'UTS02', 212),","('something_that_is_se', 'UTS02', 248),","('physical_objects_and', 'UTS02', 324),","('body_parts_and_physi', 'UTS02', 221),","('a_specific_year_in_t', 'UTS02', 63),","('physical_features_of', 'UTS02', 291),","('action_or_movement', 'UTS02', 302),","('a_specific_year_or_d', 'UTS02', 24),","('a_specific_year_in_t', 'UTS02', 460),","('a_specific_year_in_t', 'UTS02', 197),","('a_person_or_situatio', 'UTS02', 283),","('measurements_of_dist', 'UTS02', 22),","('physical_contact_or_', 'UTS02', 74),","('physical_features_or', 'UTS02', 78),","('physical_movement_or', 'UTS02', 123),","('a_specific_decade_or', 'UTS02', 114),","('physical_gestures_an', 'UTS02', 282),","('physical_contact_or_', 'UTS02', 323),","('laughter_or_amusemen', 'UTS02', 426),","('a_specific_year_or_d', 'UTS02', 470),","('physical_movement_or', 'UTS02', 287),","('food_preparation', 'UTS02', 181),","('a_sense_of_fear_or_d', 'UTS02', 140),","('laughter_or_amusemen', 'UTS02', 190),","('relationships_and_lo', 'UTS02', 314),","('body_positioning', 'UTS02', 226),","('emotional_expression', 'UTS02', 398),","('a_sudden_emotional_r', 'UTS02', 294),","('distance_or_proximit', 'UTS02', 328),","('sexuality_and_gender', 'UTS02', 462),","('identity_and_how_it_', 'UTS02', 112),","('physical_features_or', 'UTS02', 445),","('physical_contact', 'UTS02', 453),","('physical_movement_or', 'UTS02', 141),","('identity_and_culture', 'UTS02', 495),","('idea_of_something_be', 'UTS02', 252),","('outdoor_scenery', 'UTS02', 408),","('task_or_responsibili', 'UTS02', 17),","('travel_or_relocation', 'UTS02', 130),","('emotional_turmoil_an', 'UTS02', 206),","('a_specific_year_in_t', 'UTS02', 397),","('interpersonal_relati', 'UTS02', 254),","('emotion,_particularl', 'UTS02', 179),","('a_specific_year', 'UTS02', 16),","('surprise,_shock,_dis', 'UTS02', 168),","('physical_injury_or_t', 'UTS02', 158),","('physical_sensations_', 'UTS02', 376),","('a_specific_year_in_t', 'UTS02', 5),","('a_specific_year_in_t', 'UTS02', 424),","('physical_sensations_', 'UTS02', 432),","('physical_features_an', 'UTS02', 61),","('an_action_or_sound_t', 'UTS02', 209),","('feelings_and_emotion', 'UTS02', 104),","('suddenness_or_unexpe', 'UTS02', 433),","('physical_pain_and_in', 'UTS02', 157),","('outdoor_scenery', 'UTS02', 257),","('physical_contact', 'UTS02', 410),","('travel_and_relocatio', 'UTS02', 152),","('body_language_and_fa', 'UTS02', 416),","('something_or_someone', 'UTS02', 106),","('lgbtq+_issues', 'UTS02', 70),","('passage_of_time', 'UTS02', 4),","('birthdays_and_birth_', 'UTS02', 442),","('physical_changes_to_', 'UTS02', 417),","('communication_or_lac', 'UTS02', 357),","('concept_of_time_pass', 'UTS02', 464),","('a_specific_year_in_t', 'UTS02', 425),","('relationships_and_gr', 'UTS02', 258),","('relationships_and_li', 'UTS02', 9),","('emotions_and_feeling', 'UTS02', 60),","('weather_and_water', 'UTS02', 97),","('someone_or_something', 'UTS02', 275),","('a_specific_year_or_d', 'UTS02', 52),","('a_moment_in_time', 'UTS02', 337),","('life_events_and_mile', 'UTS02', 177),","('travel_and_relocatio', 'UTS02', 305),","('reaction_or_re

In [31]:
D = {
    "poly": [
        ("physical_features,_s", "UTS02", 217),
        ("something_that_is_se", "UTS02", 248),
        ("action_or_movement", "UTS02", 302),
        ("food_preparation", "UTS02", 181),
        ("a_sense_of_fear_or_d", "UTS02", 140),
        ("religion_and/or_illn", "UTS02", 322),
        ("a_moment_in_time", "UTS02", 337),
        ("action_or_activity", "UTS02", 14),
    ],
}
assert len(r[r.id_tup.isin(D['poly'])]) == len(D['poly'])

In [32]:
D_REVISED = {
    "poly": [
        ("physical appearance", "UTS02", 217),
        ("food preparation", "UTS02", 217),
        ("something negative or undesirable", "UTS02", 248),
        ("relationships", "UTS02", 248),
        ("movement", "UTS02", 302),
        ("communication", "UTS02", 302),
        ("food", "UTS02", 181),
        ("clothing", "UTS02", 181),
        ("fear", "UTS02", 140),
        ("silence", "UTS02", 140),
        ("religion", "UTS02", 322),
        ("illness", "UTS02", 322),
        ("moment in time", "UTS02", 337),
        ("movement", "UTS02", 337),
        ("location", "UTS02", 14),
        ("action", "UTS02", 14),
    ],
}
json.dump(D_REVISED, open(f"polysemantic_{subject}.json", "w"), indent=0)  # , indent=4)

with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    rp = r[r.id_tup.isin(D["poly"])]
    display(
        rp[
            [
                "id",
                "explanation_init_strs",
                "mean_score_synthetic",
                "fmri_test_corr",
                "score_synthetic",
                "top_ngrams_module_25",
            ]
        ]
    )

,id,explanation_init_strs,mean_score_synthetic,fmri_test_corr,score_synthetic,top_ngrams_module_25
805,"('physical_features,_s', 'UTS02', 217),","physical features, such as hair, fingernails, and skin\n----\nphysical features, such as hair, skin, and clothing\n----\nphysical appearance, such as hair, skin, nails, and clothing\n----\nfood preparation\n----\nphysical features or objects",0.124071,0.449662,"[0.179, 0.179, 0.093, 0.091, 0.078]","[skin stripped from, peel and cut, of skin peeled, thinly sliced, closely mown grass, rolled up sleeves, have thinly sliced, skin peeled off, long shaggy hair, back my unibrow, that many wrinkles, my unibrow, my black fingernails, long golden hair, white like foam, feet the loincloth, mown grass, bleach blonde hair, his long hair, hair and beard, visible body hair, hair and freckles, blonde hair dangling, longer and thicker, blanket and unbuttoned]"
632,"('something_that_is_se', 'UTS02', 248),","something that is seen as negative or undesirable\n----\nsomething that is unexpected, unusual, or out of the ordinary\n----\nrelationships and emotions\n----\nsomething that is either negative or controversial",0.086897,0.530197,"[0.119, 0.107, 0.093, 0.083]","[believe however tenuously, fathers feeling marginalized, seemed boldly unafraid, our vows unimpeachable, it seemed incongruous, compromise is anathema, their infinitely justifiable, seemed incongruous, became painfully intimate, drinks are unhealthy, my highly sanitized, children an incomprehensibly, said very rudely, was strangely tentative, very conception unnatural, i was overreacting, conception unnatural, all very innocent, where i exaggerated, personality any diagnosis, live is fundamentally, him semi jokingly, an almost resigned, which was farcical, relationship became painfully]"
739,"('action_or_movement', 'UTS02', 302),",action or movement\n----\ncommunication,0.076774,0.442556,"[0.106, 0.048]","[i provoked gasps, i stopped midstride, heard another thud, later the doorbell, who i googled, stalls i brace, the crop unharmed, and i braced, tell her sisters, partner michael radioed, stopped midstride, divert the asteroid, other pager buzzed, became my cellmate, already called ahead, two tables away, forward another friend, gun two inches, tell my friends, googling her googling, phone through colleagues, skype the unboyfriend, they call upstairs, told the unboyfriend, i google strangers]"
1312,"('food_preparation', 'UTS02', 181),",food preparation\n----\nobjects or actions related to food preparation\n----\nitems or activities related to food and clothing\n----\nobjects or activities related to manual labor or preparation of food,0.066000,0.433427,"[0.092, 0.08, 0.076, 0.071]","[a walrus mustache, some sliced cucumber, sliced cucumber, shirt the suspenders, piece of tomato, buttered slices, with lemon slices, the scotch tape, lemon slices, boots clanking, crinkle um skin, two by fours, lit the cigarette, throat was slit, gaffers tape, wearing flower aprons, they gaffers tape, my boots clanking, a white handkerchief, toothbrush floss, pigtails with ribbons, walrus mustache, skivvies toothbrush floss, feet the loincloth, sugar cubes]"
1088,"('a_sense_of_fear_or_d', 'UTS02', 140),","a sense of fear or danger, as many of the phrases involve fleeing, hiding, or becoming silent\n----\nsudden or unexpected silence or stillness\n----\nsilence or lack of communication",0.065678,0.521978,"[0.135, 0.078, 0.042]","[sits there silent, become very silent, room went silent, start walking faster, fled the house, leaves the room, lie down and, i crawled quietly, fell silent and, try to sneak, darted to safety, get to safety, crowd fell silent, instinct to hide, runs out crying, i stayed silent, parents were silent, i stopped midstride, she started laughing, slowly stand back, she fainted, could escape quietly, i stumbled forward, gulping for air, when she fainted]"
731,"('a_moment_in_time', 'UTS02', 337),",a moment in time\n----\nmovement or action,0.0

In [33]:
for k in D_REVISED['poly']:
    print(str(k) +': ', str(rp[rp.module_num == k[2]].iloc[0]['top_ngrams_module_50']) + ',')

('physical appearance', 'UTS02', 217):  ['skin stripped from', 'peel and cut', 'of skin peeled', 'thinly sliced', 'closely mown grass', 'rolled up sleeves', 'have thinly sliced', 'skin peeled off', 'long shaggy hair', 'back my unibrow', 'that many wrinkles', 'my unibrow', 'my black fingernails', 'long golden hair', 'white like foam', 'feet the loincloth', 'mown grass', 'bleach blonde hair', 'his long hair', 'hair and beard', 'visible body hair', 'hair and freckles', 'blonde hair dangling', 'longer and thicker', 'blanket and unbuttoned', 'hair in pigtails', 'pigtails with ribbons', 'some sliced cucumber', 'the loincloth', 'blue flower heads', 'cucumber and mayonnaise', 'swollen and red', 'sky with clouds', 'little bristly hairy', 'sliced red', 'a long white', 'black fingernails', 'sliced cucumber', 'sliced red onion', 'shave his beard', 'bristly hairy', 'long brown hair', 'big shoulder pads', 'cut the apples', 'buttered slices', 'freshly painted walls', 'torn fishnet and', 'a loincloth'

In [36]:
D_REVISED_NGRAMS = {
    ('physical appearance', 'UTS02', 217):  ['skin stripped from', 'rolled up sleeves', 'long shaggy hair', 'back my unibrow', 'that many wrinkles', 'my unibrow', 'my black fingernails', 'long golden hair'],
    ('food preparation', 'UTS02', 217):  ['peel and cut', 'thinly sliced', 'cucumber and mayonnaise', 'sliced red onion', 'cut the apples'],

    ('something negative or undesirable', 'UTS02', 248):  ['became painfully intimate', 'drinks are unhealthy', 'said very rudely', 'very conception unnatural', 'i was overreacting'],
    ('relationships', 'UTS02', 248):  ['fathers feeling marginalized', 'our vows unimpeachable', 'children an incomprehensibly', 'relationship became painfully', 'relationship became depressed'],

    ('movement', 'UTS02', 302):  ['i stopped midstride', 'stalls i brace', 'forward another friend', 'job running interference', 'pass the baton'],
    ('communication', 'UTS02', 302):  ['tell her sisters', 'already called ahead', 'tell my friends', 'phone through colleagues', 'skype the unboyfriend', 'they call upstairs', 'me her number'],
    
    ('food', 'UTS02', 181):  ['some sliced cucumber', 'piece of tomato', 'lemon slices', 'sugar cubes', 'steamed mussels'],
    ('clothing', 'UTS02', 181):  ['shirt the suspenders', 'boots clanking', 'wearing flower aprons', 'a white handkerchief', 'rolled up sleeves'],

    ('fear', 'UTS02', 140):  ['fled the house', 'darted to safety', 'instinct to hide', 'runs out crying', 'could escape quietly'],
    ('silence', 'UTS02', 140):  ['sits there silent', 'become very silent', 'room went silent', 'i crawled quietly', 'try to sneak'],

    ('religion', 'UTS02', 322):  ['like creationist biology', 'a particularly devout', 'believe however tenuously', 'in this metaphysical', 'boyish almost biblical', 'attended a prestigious', ],
    ('illness', 'UTS02', 322):  ['bipolar obsessive compulsive', 'personality any diagnosis', 'as paranoid schizophrenic', 'three month prognosis', 'suicidal depressed', 'my almost anorexically'],

    ('moment in time', 'UTS02', 337):  ['later the doorbell', 'moment later',  'suddenly he stops', 'a minute later', 'hour later he'], 
    ('movement', 'UTS02', 337):  ['neighbor came over', 'enters the room', 'he came downstairs', 'comes walking towards', 'son comes running'],
    
    ('location', 'UTS02', 14):  ['ship to hoboken', 'hotel across town', 'our small rural',  'my whole hometown', 'middlebury vermont', 'ubiquitous in suburban'], 
    ('action', 'UTS02', 14):  ['we washed dishes', 'shopped the registries', 'became a chemist', 'clean an office', 'writing his obituary'],
}

In [37]:
joblib.dump(D_REVISED_NGRAMS, f'polysemantic_ngrams_{subject}.pkl')

['polysemantic_ngrams_UTS02.pkl']